# Harvesting @ MLAI Training - Third Round Overview

At this point in the series of experiments, we're going to try some divergent techniques. I'd like to try some tactics that will improve the fit of the model to continuous use in the live application flow. Right now, the model trains on retrospective data sets and more or less expects to see full sessions. In this experiment, I'm going to try several approaches.

## Roadmap
### Feature engineering and ML algorithms

1. Instead of counting full sessions in the training datasets, we will only take sequences of a maximum N, to see if there's a useful threshold where we could evaluate sliding windows of session history. I'll compare recall at 5, 10, 15, 20 element session subsequences, offset randomly from session start. 
    - Where do the metrics drop off? 
    - Does the approach essentially require full sessions?
    - Can we query partial sessions against a model trained on full sessions?
    - Can we query partial sessions against a model trained on partial sessions?
    - Randomly drop out D% of transactions from every session.
1. Instead of using bag-of-txns, I will try a vector that contains the last N sessions by number, indexed from a transaction dictionary. This will force sequence into the model.
    - Training on the first N transactions:
        - Count N transactions from the beginning of a session.
    - Train on N-gram tiles
    - Train on N-gram shingles
1. Instead of training an XGBoost model, train neural network models that are designed to remember sequences
    - LSTM
    - Convolutional (?)
    
### Software engineering
To this point, I've mostly worked out of a some-what scattered Jupyter notebook pile of global variables, tuning up a few functions here and there to keep scopes clean. In particular, the Sagemaker code is terrible cut-and paste. As I work through the feature engineering & algorithmic roadmap, I also want to drive toward the following engineering goals:

- Establish a consistent data manipulation pipeline that is easily customizable and reentrant, with no global variables
- a set of wrapper functions for the Sagemaker API, so that I can have a simple pipeline where training jobs -> models -> batch transformations or endpoints, without giant string constant parameters that are impossible to edit or sight-check
- Automatically engage a hyperparameter tuning job as desired without changing how the model is implemented or trained.
- Easily training many models at once
- Easily launch many batch transform jobs/endpoints at once.
- Easily display the results of the many jobs at once.
- Particular feature engineering processes glide smoothly from experimental tinkering to code that can be put into production, without throwing everything away. Some of this just falls out of doing everything else better.

# Training Data
The training dataset consists of several hours of raw transaction logs containing activity from all users, with the full collection of harvesting activity from the LiquidTension harvester across two years. All LiquidTension(LT) activity is labeled as 'BadActor' = 1, while all other traffic is assumed to be innocent and labeled as 'BadActor' = 0. Since LiquidTension is currently our only easily-identified single harvester, we need his full range of activity to have a BadActor sessions in proportion to innocent sessions for training to work properly.

The training set includes the following files:

|File       |Contents                             |Rows|
------------|-------------------------------------|----|
|may1.tsv|raw transactions|119474|
|may2.tsv|raw transactions|43608|
|may3.tsv|raw transactions|30844|
|lt-only.tsv|raw transactions for a known attacker|61917|

In all, we have 193926 transactions from "innocent" sessions and 61917 LT sessions. Approximately 32% of transactions are labeled BadActor = 1, giving us a reasonable proportion in both classes.

# Sequence testing phase 1. 

## Warm up
Import standard libraries and prepare the environment.

In [42]:
import io
import os
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 

import boto3
import sagemaker
from sagemaker import get_execution_role

import toolz


%matplotlib inline

!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [2]:
# sagemaker session, role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# S3 bucket name
bucket = sagemaker_session.default_bucket()

# common column names
bad_col='BadActor'
sess_col='SessionNo'
txn_col='Act'
logtime_col = 'LogTime'



## Download
Retrieve the datafiles from the project's designated S3 bucket.

In [4]:
s3 = boto3.resource('s3')
b = s3.Bucket('sagemaker-mlai-harvesting')

# b.download_file( 'data/MLAI_ParsedDataSet.tsv', 'data/data.tsv')
b.download_file( "data/MinimalLogs/Minimal_May01.rpt", 'data/may1.tsv')
b.download_file( "data/MinimalLogs/Minimal_May02.rpt", 'data/may2.tsv')
b.download_file( "data/MinimalLogs/Minimal_May03.rpt", 'data/may3.tsv')
b.download_file( "data/MinimalLogs/Minimal_OnlyLT.rpt", 'data/lt-only.tsv')


may1 = pd.read_csv('data/may1.tsv',sep='\t')
may2 = pd.read_csv('data/may2.tsv',sep='\t')
may3 = pd.read_csv('data/may3.tsv',sep='\t')
lt = pd.read_csv('data/lt-only.tsv',sep='\t')



txn = may1.append([may2, may3, lt])
txn[logtime_col] = pd.to_datetime(txn[logtime_col])
# txn[txn[bad_col]==1]

## Data conversion and feature engineering
In real life, a session consists of a series of rows of transactions of different types, and each transaction type records a variable number of additional metadata attributes describing a logged event, for a total of over 30 columns of extracted data. In addition, our tagging process has given each row a BadActor label.

|sessionno|txn id|BadActor|parm1|parm2|...|
|---------|------|--------|-----|-----|---|
|1240|111|0|query string|...|...|
|1240|112|0|meta|...|...|
|2993|301|1|meta|...|...|


In [4]:
# 'Innocent' log entries
txns = pd.DataFrame(np.sort(txn['Act'].unique()))

# Harvesting log entries
lt_txns = pd.DataFrame(np.sort(lt['Act'].unique()))



We drop most of this information, including the temporal sequence of the log entries, and convert each session into a single row of data. Almost all of the columns go away, replaced by counts of transaction types in the session.

|sessionno|BadActor|111|112|113|...|301|302|...|
|---------|--------|---|---|---|---|---|---|---|
|1240|0|1|1|0|...|0|0|...|
|2993|1|0|0|0|...|1|0|...|

# Truncating sessions
Before we flatten the sessions, we're going to truncate them. This may be a better match for the real world, in which at best we will be able to scan sliding windows of transactions with a scaling resumption that we may not scan every event.

We'll try several approaches at once:
- dropping out D% of transactions from every session
- taking only the first N sessions from every session
- taking on the last N transactions from every session
- taking N consecutive transactions from the middle of every session. 
- choosing N transactions as above, but dropping every session without at least N transactions.

In [37]:
def drop_column_groups( txn_c ):
    txn_c.columns = txn_c.columns.droplevel(0)  
    txn_c.rename_axis(None, axis=1).reset_index()
    return txn_c

def get_session_groups( txn ):
    txn_g = txn.groupby(sess_col)
    return txn_g

txng = get_session_groups(txn)
txng

In [47]:
%%time
def drop_pct( df, n = .1 ):
    return df.sample(frac= 1-n)

def first_n( df, n = 5):
    return df.head( n )
    
def last_n( df, n = 5):
    return df.tail( n )


In [84]:
def grappl( fun, parm ):
    return lambda df: df.apply(fun, parm)


In [127]:
def flatten_txns( txn_log ):
    txn_narrow = txn_log[[sess_col, txn_col,bad_col]]
    txn_pivot = pd.pivot_table(txn_narrow, index=[sess_col,bad_col], columns = [txn_col],aggfunc=[len]).fillna(0)
    txn_pivot.columns = txn_pivot.columns.droplevel(0)           # the pivot table has a two-level index
    txn_flat = txn_pivot.rename_axis(None, axis=1).reset_index() # these two lines get rid of it so we have a simple table
    return txn_flat

In [59]:
flats = [flatten_txns( txn.reset_index() ) for txn in txngs]

In [77]:
jobs = [['drop10', [drop_pct, .1]], 
        ['first5', [first_n, 5]],
        ['last5', [last_n, 5]]
       ]

[(name, func, parm) for [name, [func, parm]] in jobs]

[('drop10', <function __main__.drop_pct(df, n=0.1)>, 0.1),
 ('first5', <function __main__.first_n(df, n=5)>, 5),
 ('last5', <function __main__.last_n(df, n=5)>, 5)]

In [131]:
def prep_jobs(df, jobs):
    groups = [[name, grappl(fun, parm)(df)] for [name, [fun, parm]] in jobs]
    flats = [[name,flatten_txns( txn.reset_index() )] for [name,txn] in groups]
    return flats

In [132]:
%%time

gs = prep_jobs(txng, jobs )


CPU times: user 38.6 s, sys: 232 ms, total: 38.8 s
Wall time: 38 s


In [134]:
txngs= gs
[(name,len(df)) for [name,df] in txngs]
gs[0]

['drop10',
         SessionNo  BadActor  111  112  114   115  116  117  118  119  ...  \
 0     -2147481927         0  1.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...   
 1     -2147360137         1  1.0  1.0  0.0   0.0  0.0  0.0  0.0  0.0  ...   
 2     -2147317281         0  1.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...   
 3     -2147002735         0  1.0  0.0  0.0   2.0  1.0  0.0  0.0  0.0  ...   
 4     -2146953899         0  0.0  1.0  0.0  19.0  0.0  0.0  0.0  0.0  ...   
 5     -2146926264         0  1.0  0.0  0.0   1.0  0.0  1.0  0.0  0.0  ...   
 6     -2146915841         0  1.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...   
 7     -2146723372         0  1.0  0.0  0.0   0.0  0.0  1.0  0.0  0.0  ...   
 8     -2146089473         0  1.0  0.0  0.0   1.0  1.0  1.0  0.0  0.0  ...   
 9     -2145757832         0  1.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...   
 10    -2145629827         1  1.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...   
 11    -2145548627         1  1.0  0.0  0.0   0.0  0.

## Producing pools of training and testing data

In order to support simultaneous execution of multiple jobs, this notebook introduces a new scheme for piping data through to models.

The normal flow runs as follows:

input data(s3) -> df's on notebook instance -> train.csv, test.csv, validate.csv on notebook instance -> s3/out/ -> Sagemaker instances

Hardcoding these filenames is fine for playing around in a notebook, but it limits us to one job at a time.

In this approach, every job has a base Name. This name will carry through from S3 into the Sagemaker instances.
Training data files will reside in `<s3bucket>/<key>/out/Name`.

As before, in each `Name` subfolder, we will divide the combined good and bad data pools as follows:
- a training set that the model iterates over during the learning process
- a test set that is used to evaluate the model during training
- a validation set that is kept separate to test the model after training is complete. We need separate test and validate pools in order to make sure that we're not overfitting the model to a single set of test data.


In [233]:
class FrameSplitter:
    def __init__(self, y_col, ignore_cols):
        self.y_col = y_col
        self.ignore_cols = ignore_cols

    def reformat( self, df ):
        y = df[self.y_col]
        csv = pd.concat( [y, df.drop(self.y_col,axis=1).drop(self.ignore_cols,axis=1)], axis=1)
        return csv

    def map_sets( fun, dfs ):
        '''
        fun - function that operates on a df
        dfs - [[name1, df1], [name2,df2],...]
        
        returns - [result1, result2,...]
        '''
        
        return [[name,fun(df)] for [name, df] in dfs]
    
    def to_csv( self, df, file, sepr=','):
        '''
        Write a CSV that meets Sagemaker's requirements for training instances:
            - y columns are at [0]
            - non-x, non-y columns are dropped so they don't affect training
        '''
        df.to_csv( path_or_buf=file, sep=sepr,index=False, header=False)
    
    def split_frame( self, df, train_frac ):
        '''
        df - DataFrame (or array) containing rows of data for training, testing, and validation
        train_frac - Decimal representing how much of the set should be used for training.
        After reserving train_frac * len(df) rows for training, divide the remainder in half for testing and validation 
        populations.
        returns - list of dataframes as [training, testing, validation]
        '''
        l = len(df)
        test_frac = (1-train_frac)/2
        tr = int(train_frac * l)
        te = int(tr + test_frac * l)

        train = df[:tr]
        test = df[tr:te]
        val = df[te:]
        return [train, test, val]
    
    def train_split( self, df, y_split=.8 ):
        '''
        df - dataframe of training data with one labeled column
        y_col - column containing labels 1 or 0
        Given a dataset with rows in two classes, distinguished by a column y_col with values 1 or 0,
        split the dataset into [train, test, validate] sets containing values from both classes.       
        '''
        y1 = df[df[self.y_col]==1]
        y0 = df[df[self.y_col]==0]

        y1 = split_frame(y1, y_split)
        y0 = split_frame(y0, y_split)

        dfs = []
        for i in range(3):
            # Dropping the session # because we don't want to train on it.
            # Also leaves our label - BadActor - in the 0 column, as XGBoost requires for CSV
            dfi = y1[i].append(y0[i]).sample(frac=1)
            dfs.append( dfi )
    
        return dfs
    
#     def write_splits( splits, path, sepr = ',', ext='csv'):
#         '''
#         splits - list of three dfs to be written as [train.ext, test.ext, validate.ext]
#         path - path to write files to. Directory need not exist
#         sepr - separator char for output files - typically , or \t
#         ext - file extension for output files, typically csv or txv
#         Write train, test, validate delimited files to specified subfolder. Ensure that the subfolder exists.
#         returns - filenames with full paths
#         '''
        
#         filenames = [n + ext for n in ['train.', 'test.', 'validation.']]
        
#         #TODO - make the dir, write the files
        
        
#         return split_files
        
    
    def train_splits( self, dfs, split=0.4, sepr=',', ext='csv'):
        '''
        dfs - list of named dfs as [name, df] pairs
        sepr - separator char for output files - typically , or \t
        ext - file extension for output files, typically csv or txv
        Split a list of named dfs into train, test, validate csv's organized into subdirectories by the df names
        returns - list of output filenames in [train, test, validate] triples, in same order as source files in dfs
        '''
        files = []
        files = [self.train_split (df, split) for [name, df] in dfs]
#         files = [write_splits (df, name, sepr, ext) for [name, df] in dfs]
        
        return files
    

In [234]:
csv = FrameSplitter( bad_col, [sess_col])
txn_narrow = FrameSplitter.map_sets( csv.reformat, gs )
trains = csv.train_splits(txn_narrow)
csv.to_csv( trains[0][0], "test.csv")
# trains[0][0].to_csv(sep=',')

In [110]:
def split_frame( df, train_frac ):
    l = len(df)
    test_frac = (1-train_frac)/2
    tr = int(train_frac * l)
    te = int(tr + test_frac * l)
    
    train = df[:tr]
    test = df[tr:te]
    val = df[te:]
    return [train, test, val]

In [111]:
def train_split( flat, bad_split=.8 ):
    bad = flat[flat[bad_col]==1]
    good = flat[flat[bad_col]==0]
    
    bads = split_frame(bad, bad_split)
    goods = split_frame(good, bad_split)
    
    dfs = []
    for i in range(3):
        # Dropping the session # because we don't want to train on it.
        # Also leaves our label - BadActor - in the 0 column, as XGBoost requires for CSV
        df = bads[i].append(goods[i]).drop(sess_col,axis=1).sample(frac=1)
        dfs.append( df )
    
    return dfs
    


# Split the data and upload to S3
Break the set into train, test, and validation collections and output CSV's.
As Sagemaker requires, leave out row indices and column headers.

In [113]:
# dfs = train_split(base, .4)
# train_splits = [(name, train_split(df, .4)) for [name, df] in txngs]


In [117]:
[name,df]=txngs[0]

df.reset_index()

,SessionNo,level_1,LogTime,CustID,GroupID,ProfID,Act,BadActor
0,-2147481927,78744,2019-05-01 22:09:42,s9003448,main,eds,111,0
1,-2147360137,59038,2019-04-23 00:41:52,s8986718,main,iconnweb,112,1
2,-2147360137,59039,2019-04-23 00:41:52,s8986718,main,iconnweb,311,1
3,-2147360137,59037,2019-04-23 00:41:39,s8986718,main,iconnweb,111,1
4,-2147317281,118033,2019-05-01 23:55:06,s8989685,main,edsapi,121,0
5,-2147317281,118446,2019-05-01 23:56:29,s8989685,main,edsapi,121,0
6,-2147317281,117986,2019-05-01 23:54:56,s8989685,main,edsapi,111,0
7,-2147317281,118004,2019-05-01 23:54:59,s8989685,main,edsapi,121,0
8,-2147002735,28950,2019-05-04 19:52:22,s8989984,main,ehost,121,0
9,-2147002735,29635,2019-05-04 19:54:52,s8989984,main,ehost,116,0


In [256]:
!mkdir out

s3_client = boto3.client('s3')
bucket = "sagemaker-mlai-harvesting"

for i, df in enumerate(dfs):
    files = ["train","test","validate"]
    file = "out/{}.csv".format(files[i])
    df.to_csv(path_or_buf= file, header=False, index=False  )

    print("Uploading {} to {}".format(file, bucket))

    response = s3_client.upload_file(file, bucket, file)
    print(response)
    
    
    

Uploading out/train.csv to sagemaker-mlai-harvesting
None
Uploading out/test.csv to sagemaker-mlai-harvesting
None
Uploading out/validate.csv to sagemaker-mlai-harvesting
None


# Prepare and train a model
Boilerplate code mostly copied from Amazon sample code at https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/xgboost_abalone/xgboost_abalone.ipynb, with ample room for improvement.

In [257]:
%%time
region = 'us-east-1'
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(region, 'xgboost')


from time import gmtime, strftime

job_name = 'harvesting-xgboost-binary-classification' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Training job", job_name)

#Ensure that the training and validation data folders generated above are reflected in the "InputDataConfig" parameter below.

create_training_params = \
{
    "AlgorithmSpecification": {
        "TrainingImage": container,
        "TrainingInputMode": "File"
    },
    "RoleArn": role,
    "OutputDataConfig": {
        "S3OutputPath": os.path.join("s3://",bucket, "out", "xgb-class") 
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.m4.4xlarge",
        "VolumeSizeInGB": 5
    },
    "TrainingJobName": job_name,
    "HyperParameters": {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "silent":"0",
        "objective":"binary:logistic",
        "num_round":"50"
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 3600
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://sagemaker-mlai-harvesting/out/train.csv" , 
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "text/csv",
            "CompressionType": "None"
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://sagemaker-mlai-harvesting/out/validate.csv" ,
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "text/csv",
            "CompressionType": "None"
        }
    ]
}


client = boto3.client('sagemaker', region_name=region)
client.create_training_job(**create_training_params)

import time

status = client.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
print(status)
while status !='Completed' and status!='Failed':
    time.sleep(60)
    status = client.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
    print(status)

Training job harvesting-xgboost-binary-classification2019-06-21-19-33-25
InProgress
InProgress
InProgress
Completed
CPU times: user 77.3 ms, sys: 4.05 ms, total: 81.4 ms
Wall time: 3min


In [260]:
%%time
import boto3
from time import gmtime, strftime

model_name="harvesting-xgboost-binary-class2019-06-21-19-33-25"+ '-model'
print(model_name)

info = client.describe_training_job(TrainingJobName=job_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)

primary_container = {
    'Image': container,
    'ModelDataUrl': model_data
}

create_model_response = client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

harvesting-xgboost-binary-class2019-06-21-19-33-25-model
s3://sagemaker-mlai-harvesting/out/xgb-class/harvesting-xgboost-binary-classification2019-06-21-19-33-25/output/model.tar.gz


ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Cannot create already existing model "arn:aws:sagemaker:us-east-1:872344130825:model/harvesting-xgboost-binary-class2019-06-21-19-33-25-model".

In [261]:
from time import gmtime, strftime

endpoint_config_name = 'Harvest-XGBoostEndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])


Harvest-XGBoostEndpointConfig-2019-06-21-19-39-54
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:872344130825:endpoint-config/harvest-xgboostendpointconfig-2019-06-21-19-39-54


# Launch an endpoint

In [262]:
%%time
import time

endpoint_name = 'Harvest-XGBoostEndpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

resp = client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

while status=='Creating':
    time.sleep(60)
    resp = client.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Status: " + status)

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)



Harvest-XGBoostEndpoint-2019-06-21-19-40-24
arn:aws:sagemaker:us-east-1:872344130825:endpoint/harvest-xgboostendpoint-2019-06-21-19-40-24
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-1:872344130825:endpoint/harvest-xgboostendpoint-2019-06-21-19-40-24
Status: InService
CPU times: user 142 ms, sys: 283 µs, total: 142 ms
Wall time: 8min 1s


# Test the model
Currently, we launch an endpoint to test the model. This endpoint includes a simple web service that takes POST request with rows of or model's X values - columns other than BadActor - and returns a corresponding list of Y values - BadActor predictions.

The endpoint approach is most suitable to interactive use, such as possibly using the model to blacklist a harvesting session as soon as it is identified. For offline analysis, this should be reconfigured to run batch transform jobs instead, which are cheaper to run and more streamlined to invoke.

In [263]:
runtime_client = boto3.client('runtime.sagemaker', region_name=region)

import json
from itertools import islice
import math
import struct

!head -10000 out/test.csv > out/single-test.csv

file_name = 'out/single-test.csv' 

# file_name = "out/may8.csv"


csv = pd.read_csv(file_name, header=None)
csv.columns
label = csv[0]
csv = csv.drop(0,axis=1)

single = "out/single.csv"

csv.to_csv(path_or_buf=single, header=False, index=False)

with open(single, 'r') as f:
    payload = f.read().strip()

In [264]:
response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='text/csv', 
                                   Body=payload)
result = response['Body'].read()
result = result.decode("utf-8")
result = result.split(',')
result = [round(float(i)) for i in result]


# Compute the confusion metrics

A confusion matrix describes the proportions of true and false positives and negatives, together with some derived metrics.

In [265]:
comp = pd.concat( [label, pd.DataFrame(result)], axis = 1)
comp.columns =["label",'prediction']

label_positive = comp['label'] == 1
predict_positive = comp['prediction'] == 1

tp = len( comp[label_positive & predict_positive])
fp = len( comp[~label_positive & predict_positive])
tn = len( comp[~label_positive & ~predict_positive])
fn = len( comp[label_positive & ~predict_positive])
m = len(comp)

accuracy = (tp+tn)/m
precision = tp/(tp+fp)
recall = tp/(tp+fn)

print("accuracy: {} precision: {} recall {}".format(accuracy, precision,recall))

accuracy: 0.968754320475598 precision: 0.9665454545454546 recall 0.8807157057654076


In [266]:
tp,fp,tn,fn, len(comp)

(1329, 46, 5678, 180, 7233)

# V1 results
accuracy: 0.9657853810264385 precision: 1.0 recall 0.8360655737704918

|tp|fp|tn|fn|len(comp)|
|--|--|--|--|---------|
|1683| 0| 7632| 330|9645|

The very first time we ran the model, we achieved strikingly successful rates of harvesting identification.
The most significant number here is the recall of 84%, meaning that we successfully identified 84% of all harvesting sessions by looking only at counts of transaction types.


# V2 results, with txns + wait stats:
accuracy: 0.968754320475598 precision: 0.9665454545454546 recall 0.8807157057654076

|tp|fp|tn|fn|len(comp)|
|--|--|--|--|---------|
|1329| 46| 5678| 180|7233|

In the second run, we see an increased recall, at the price of a small number of false positives. 

# Next steps

## Further investigating the data 

We had additional ideas for modeling the data while staying in this bag-of-transaction technique.
1. Try some hyperparameter tuning to seem if the success rates can be trivially improved.
1. Enrich the training data set in various ways - add colums to summarize total session time, average time/request, and so on.
1. Perform some clustering analysis to try to identify common patterns of behavior other than LT. This may reveal the presence of other kinds of harvesting.

## Qualifying the approach
Can we use this approach to identify and blacklist harvesting sessions as they occur? Some notes:
1. The approach must be resilient to easy efforts to evade. Does the accuracy of the identification drop if the attacker makes minor changes to his workflow?
1. How long does it take to identify an attacker in real time? 
    1. Do we gain certainty soon enough to stop an attacker before he's done what he came to do?
    2. Can we tag sessions accurately after the first N log entries, for instance?
    
## Designing an implemetation
Design an architecture for identifying and intercepting harvesting activity in real time. Confirm data sources, manage impact to usage latency, model costs and ROI.

In today's world, it would be less effective to perform real-time analysis on AWS, since all of our current content usage is on-prem. The algorithm used here, XGBoost, is performant on commodity hardware, so we may be able to run on standard VMs.

In real-time analysis, we will face a stream of events from interleaved sessions. We will have to demultiplex these into individual event streams both for training and for prediction, implying some kind of windowing to capture and send sets of log entries as partial sessions. It's not clear how big the impact of this windowing will be on the accuracy of the models.

# Other analytical techniques
While this algorithm seems promising, we're throwing away a huge amount of intelligence before we start training, in the name of simplicity. We can evaluate what kind of gains we could achieve through more advanced techniques:
- Stateful models like LSTM or CNN
- more 
